<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Examples.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Examples: Multiple linear regression – advanced regression analysis 1

© ExploreAI Academy

In this notebook, we will delve deeper into multiple linear regression. We employ statistical tests and diagnostics from the Statsmodels library to assess model quality comprehensively. Using the mtcars dataset, we explore linearity and multicollinearity, as well as fit a model, enriching our regression analysis toolkit.

## Learning objectives

By the end of this notebook, you should be able to:
* Understand the importance of statistical diagnostics in regression analysis.
* Perform exploratory data analysis on a regression dataset.
* Check for linearity between predictor and response variables.
* Identify and address multicollinearity issues in the dataset.
* Fit a multiple linear regression model using Statsmodels and interpret its summary 


## Introduction

Previously, we explored the use of `sklearn` and multiple predictors to fit a regression model. When it comes to regression, there aren't many metrics and tools available in `sklearn` that allow us to assess the quality of our fitted models. Moreover, we require further techniques that will enable us to validate the characteristics of our initial data prior to conducting any modelling.  

In order to more accurately assess a regression model's quality, we go over a thorough set of methods that centre on statistical testing. This train is an adaptation of [this Medium article](https://towardsdatascience.com/how-do-you-check-the-quality-of-your-regression-model-in-python-fa61759ff685).

## The dataset: `mtcars`

For this train, we continue to make use of the `mtcars` dataset, to which we've already been introduced. We begin by importing some libraries, which will help us load and explore our data. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Load the data
df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/mtcars.csv', index_col=0)
df.head(10)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
model,,,,,,,,,,,
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [ ]:
#Explore the data
df.describe().T

It appears both `vs` and `am` are categorical variables. From the dataset description [available here](https://rpubs.com/neros/61800), `vs` takes on a value of 0 or 1 depending on whether the engine layout is a *V* or *straight*. `am` also takes on a value of 0 or 1 depending on whether the transmission is *automatic* or *manual*.

Note that these don't need to be one-hot encoded because they are binary categorical variables (only two categories each), and they already make use of zeros and ones.

## Checking for linearity

The first step in our analysis involves assessing the mathematical relationship between each predictor variable and the response variable. Linearity is a crucial aspect that we need to evaluate. A linear relationship implies that the change in the response variable *Y*, resulting from a one-unit change in the predictor $X_j$, remains consistent across different values of $X_j$.

If we attempt to fit a regression model to a dataset that exhibits non-linear patterns, the model may inadequately capture the underlying relationships within the data, leading to inaccuracies in predictions. To assess linearity, we often create scatter plots for each predictor against the response variable. This visual examination helps us identify any apparent linear trends in the data.


In [ ]:
fig, axs = plt.subplots(2,5, figsize=(14,6),)
fig.subplots_adjust(hspace = 0.5, wspace=.2)
axs = axs.ravel()

for index, column in enumerate(df.columns):
    axs[index-1].set_title("{} vs. mpg".format(column),fontsize=16)
    axs[index-1].scatter(x=df[column],y=df['mpg'],color='blue',edgecolor='k')

fig.tight_layout(pad=1)

It appears that a significant amount of our variables have an approximately linear relationship. Here, a trend between each of `disp, hp, drat, wt`, `qsec`, and `mpg` seems to exist.

Based on these findings, it appears that there is enough linearity present to apply a linear regression model. Let us proceed.

We'll make a duplicate of the dataset to experiment with and process it with `statsmodels.OLS()`, the least squares regression module in the `statsmodels` library. We'll use this module to perform our regression.

In [ ]:
df1 = df.copy()

## Checking for multicollinearity

Multicollinearity refers to the presence of a strong correlation among two or more of the predictor variables in the dataset. The presence of any correlation among predictors is detrimental to model quality for two reasons:

1.  It tends to increase the standard error of the coefficients estimates, making them less precise and leading to wider confidence intervals.

2. It becomes difficult to estimate the effect of any one predictor variable on the response variable because multicollinearity makes the coefficients sensitive to small changes in the model or the data, which can lead to unstable coefficient estimates.

We will test for multicollinearity by creating paired scatter plots of predictors and then creating a correlation heatmap.

### Pairwise scatter plots

As can be inferred by the name, a pairwise scatter plot matrix simply produces an $n \times n$ matrix, where $n$ is the total number of variables compared. Each cell in this matrix represents the relationship between two variables. The diagonal cells of this visual matrix represent the comparison of a variable with itself. However, instead of displaying a scatter plot comparison, these diagonal cells typically contain a histogram or a density plot representing the univariate distribution of each variable. This allows for visualising the distribution of values taken by each variable.


In [ ]:
# Due to the number of visuals created, this codeblock takes about one minute to run.
from seaborn import pairplot
g = pairplot(df1.drop('mpg', axis='columns'))
g.fig.set_size_inches(9,9)

### Correlation heatmap

Another way we can visually discover linearity between two or more variables within our dataset is through the use of a correlation heatmap. Similar to the pairwise scatter plot we produced above, this visual presents a matrix in which each row represents a distinct variable, with each column representing the correlation between this variable and another one within the dataset. Remember, when the absolute value of the correlation coefficient is close to one, it indicates a strong linear relationship between the variables.  

In [ ]:
# We only compare the predictor variables, and thus drop the target `mpg` column.
corr = df1.drop('mpg', axis='columns').corr()

In [ ]:
from statsmodels.graphics.correlation import plot_corr

In [ ]:
fig=plot_corr(corr,xnames=corr.columns)

From both the pairwise scatterplot and correlation heatmap, we can see several strong correlations among predictors.

Let's keep these in mind as we continue to check the quality of our model.

## Fitting the model using `statsmodels.OLS`

As previously stated, `sklearn` has limited metrics and tools for evaluating the suitability of the regression models we fit. Thus, to broaden our scope, we import the `statsmodels` library, which contains a diverse range of statistical methods.

In [ ]:
import statsmodels.formula.api as sm

### Generating the regression string

Those who are familiar with the R language will know that fitting a machine learning model often involves specifying the relationship between variables using a formula interface. In this interface, a string of the form:

`y ~ X`

is commonly used, where:
- *y* represents the response variable.
- $X$ represents one or more predictor variables.

The string is read as "Regress *y* on $X$", indicating that we want to model the relationship between the response variable *y* and the predictor variable(s) $X$.

Similarly, in Python's `statsmodels` library, the formula interface is used for specifying regression models. We need to generate an appropriate string following this pattern to specify the model when fitting a regression model using the formula interface. For example, *y* ~ $X_1$ + $X_2$ represents a model where *y* is regressed on predictor variables $X_1$ and $X_2$.


In [ ]:
formula_str = df1.columns[0]+' ~ '+'+'.join(df1.columns[1:])
formula_str

In other words, we will regress `mpg` on all of the predictors.

### Construct and fit the model

We now go ahead and fit our model. We use the `ols` or Ordinary Least Squares regression model from the `statsmodels` library to do this:  

In [ ]:
model=sm.ols(formula=formula_str, data=df1)

In [ ]:
fitted = model.fit()

### Print model summary

Unlike the `sklearn` models we've seen so far which only produce a couple of statistics following the fitting process, our new model produces a rich set of statistics to help us analyse its appropriateness.   

In [ ]:
print(fitted.summary())

The above result provides the summary output of the multiple linear regression model fitted to the data.

On the top of the summary report, we'll find the **Model information section**, which provides details about the type of model utilised and the estimation method employed. Following this, the **Model fit** section offers crucial metrics such as R-squared and adjusted R-squared, providing insights into how well the model fits the data. 

Additionally, the F-statistic, accompanied by its associated p-value, evaluates the overall significance of the model. Moving further down, the **Coefficients table** displays estimates, standard errors, t-values, and p-values for each predictor variable, elucidating their individual significance and impact on the dependent variable. 

Lastly, we'll encounter additional diagnostics, encompassing various diagnostic tests aimed at assessing the model's assumptions and identifying potential issues like multicollinearity or non-normality of residuals. This succinct summary encapsulates the most salient components of a stats model summary report, offering a comprehensive overview of the model's performance and reliability.



#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>